# A notebook to develop the script, generate and analyse some results.

I have started the development of the program in this notebook. Then, I have adapted the code to have a script



In [ ]:
#!/usr/bin/env python
'''
Created on 

@author: manuel.dominguezbecerra@nhs.net

'''

# Import libraries 
import argparse                          # pip install argparse
import pandas as pd                      
import os
import numpy as np

# To allow the user to load the file after running "python run.ppy"

# Example:    python run.py -- /path/file_name1 -- /path/file_name2

   ### 1. Load the files
    
# During the development of this application, I have needed to run the script many times
# To do this in an easy way, I directly introduce here the path/file_name in a variable
# As it can be seen below

Name_file1 = "/Users/monkiky/Desktop/VCF-matcher/Twin_brothers/W1717693_S1.vcf"

Name_file2 = "/Users/monkiky/Desktop/VCF-matcher/Twin_brothers/W1816869_S14.vcf"

# In the script, the previous two lines doesn´t appear and the following lines are incommeneted

#parser = argparse.ArgumentParser()
#parser.add_argument('--vcf1', type=str, required=True)
#parser.add_argument('--vcf2', type=int, required=True)
#args = parser.parse_args()
#Name_file1 = args.vcf1 
#Name_file2 = args.vcf2

# This 6 lines allow the user to select the two neccesary files to run the script

#During the development, the few lines make running the script easier.
# In the run.py script, the previous lines are uncommented and the following lines are deleted.


# From here to the end, the following lines are identical that the one find in the script


   ### 2. Take the body of the files and ignore meta-data lines

# Originally what I did was to delete the metedata and save the data in a new file
# This approach was over complicated and doesn´t work in window machines.
# Instead of this, the current approach  take directly the data and save it in pandas dataframe object.
# This is also a more efficient method.

# For the file number 1
with open(Name_file1, 'r') as f:
    for line in f:
        if line.startswith('#') and len(line)>2 and line[1] != '#':
            columns = line[1:-1].split('\t')
            break

dataA = pd.read_csv(Name_file1, comment='#', delimiter='\t', names=columns)

# Same for the second file
with open(Name_file2, 'r') as f:
    for line in f:
        if line.startswith('#') and len(line)>2 and line[1] != '#':
            columns = line[1:-1].split('\t')
            break
            
dataB = pd.read_csv(Name_file2, comment='#', delimiter='\t', names=columns)


# If the data frame has more than 10 columns
# that means there are more than 1 sample in the vcf file
# If more than one file is found, next lines ask to the user what sample to take.

if len(dataA.columns) > 10:
     Name_sample1 = input('It seems that {} has more than one sample. Please introduce the name of the sample you wish to analyse:  '.format(os.path.basename(Name_file1)))
else:
    print('File', os.path.basename(Name_file1), 'contains one sample only.')
    Name_sample1 = dataA.columns.tolist()[9]

if len(dataB.columns) > 10:
     Name_sample2 = input('It seems that {} has more than one sample. Please introduce the name of the sample you wish to analyse:  '.format(os.path.basename(Name_file2)))
else:
    print('File', os.path.basename(Name_file2), 'contains one sample only.')
    Name_sample2 = dataB.columns.tolist()[9]

    

# The data of the sample is save in the file in one column separated by ":". 
# Example: Example:   0/1:100:3501,254:0.0676:20:-100.0000:100

# To take the data I want, I need to split and put the fiels in columns


tmp_df2 = dataA[Name_sample1].str.split(':', expand=True)

#tmp_df2 = tmp_df2.fillna(value=np.nan)
#tmp_df2 = tmp_df2.dropna(axis=1, how='any')

tmp_df2.columns = dataA.FORMAT.iloc[0].split(':')
dataA = pd.concat([dataA, tmp_df2], axis=1)

# Results so far
#  CHROM       POS ID REF ALT  QUAL FILTER   GT  GQ       AD      VF  NL
#  chr1  36931745  .   G   A  34.0     SB  0/1  34  1755,34  0.0190  20

# Same for dataB
tmp_df3 = dataB[Name_sample2].str.split(':', expand=True)
tmp_df3 = tmp_df3.fillna(value=np.nan) 
tmp_df3 = tmp_df3.dropna(axis=1, how='any') 

tmp_df3.columns = dataB.FORMAT.iloc[0].split(':')
dataB = pd.concat([dataB, tmp_df3], axis=1)



    ### 3. Filter the variants  
    
# Two filters are applied here. First, I take only the variants that "PASS" the "FILTER" field. 
# This is a mandatory field.

# Then, I take only the variants which Variant Frecuency (VF) is highter than. 0.4
# This is not a mandatory field in vcf filed

# VF is in the 9th column with another data. 
#we need to extract and put VF data in an independent column first


# Now, let´s filter the variant
# We dont want variants that doesnt pass the Filter status
# This field is mandatory, no condition is needed because it is applied to any type of VCF file.
dataA = dataA.query('FILTER == "PASS"')
dataB = dataB.query('FILTER == "PASS"')
                    
# Now, I apply Variant Frecuency (VF) filter (non-mandatory field)

# If there is a VF Genotype fields in the file
# Take the variants that are > 0.4
# If not, no VF filter is applied

if "VF" in dataA.columns:
    dataA["VF"] = dataA["VF"].astype(float)
    dataA = dataA.query('VF > 0.4')
    
if "VF" in dataB.columns:
    dataB["VF"] = dataB["VF"].astype(float)
    dataB = dataB.query('VF > 0.4')
    

   ### 4. Variants comparison

# Two variants are the same if  CHROM, POS, REF, ALT and GT are equal

dataA['CHROMPOSREFALT']= dataA["CHROM"].apply(str)+"."+dataA["POS"].apply(str)+dataA["REF"]+"."+dataA["ALT"]

dataB['CHROMPOSREFALT']= dataB["CHROM"].apply(str)+"."+dataB["POS"].apply(str)+dataB["REF"]+"."+dataB["ALT"]


frames = [dataA[['CHROMPOSREFALT', 'GT']],dataB[['CHROMPOSREFALT', 'GT']]]
semi_final_df = pd.concat(frames)
# semi_final_df saves the CHROM, POS, REF, ALT in the first column and the GT in the second column


#Example:
'''
CHROMPOSREFALT     GT

chr1.123456T.C     0/1       From sample 1
chr2.123456A.C     1/1       From sample 1
chr1.123456T.C     0/1       From sample 2

'''


# Let´s reduce the number of rows
final_df=(semi_final_df.assign(key=semi_final_df.groupby('CHROMPOSREFALT').cumcount())
      .pivot('CHROMPOSREFALT','key','GT')
      .rename(columns=lambda x:f"Sample{x+1}")
      .rename_axis(columns=None).reset_index())

# Final_df contains all common and non-common variants that passed the filters

# The pivot command gives NaNs in the column of the second sample when a variant is only found in one of the samples
# This NaN values return errors in the following lines. This is an error I have found when the script was developed
# To solve the issue without introducing many changes, NaN values are replace for 99/99 values.
final_df = final_df.replace(np.NaN,"99/99" )

   ### 4. See how many GT match

# Variables needed for both type of reports

r0 = os.path.basename(Name_file1) # Get the  file name of a path/file_name 
r1 = Name_sample1                 # Get the name of the sample 1
r2 = os.path.basename(Name_file2)
r3 = Name_sample2

if "Sample2" not in final_df.columns:
    # If Sample2 is not in final_df, that means there is not common position between the samples.
    report0 = '''
 _____________________________  REPORT  ________________________________________ 

vcf 1: {0}  AND its sample name: {1}  
vcf 2:  {2}  AND its sample name: {3}

   No common positions found between samples

 ____________________________ END REPORT  _______________________________________
'''
    print(report0.format(r0,r1,r2,r3,))

else:    
    # else match common position and carry on the report.
    final_df["Matches"] = np.where(final_df["Sample1"] == final_df["Sample2"], True, False)
    final_df.columns = ['CHROMPOSREFALT',os.path.basename(Name_file1),os.path.basename(Name_file2),"Matches" ]

    # 6.  To get hom and het 

    final_df['Genotype1'] = final_df.iloc[:,2].apply(lambda x: x.split('/' or '|')[0])
    final_df['Genotype2'] = final_df.iloc[:,2].apply(lambda x: x.split('/' or '|')[1])
    
    final_df['Hom/het'] = np.select([final_df['Matches'] & final_df['Genotype1'].eq(final_df['Genotype2']),
                       final_df['Matches'] & final_df['Genotype1'].ne(final_df['Genotype2'])],
                       choicelist=["Hom", "Het"],
                       default=pd.NA)

   ### 6. Generate results in the terminal

# Variable to introduce in the report :
    
    r4 = final_df['Matches'].value_counts().get(True, 0) # Count trues if any, retunr 0
    r5 = final_df['Hom/het'].value_counts().get("Hom", 0)
    r6 =final_df['Hom/het'].value_counts().get("Het", 0)
    r7 = final_df['Matches'].value_counts().get(False, 0)
    r8 = len(final_df)
    r9 = r4/(r4+r7)
    report = '''
 _____________________________  REPORT  ________________________________________ 

vcf 1: {0}  AND its sample name: {1}  
vcf 2:  {2}  AND its sample name: {3}

                                                  Homozigous: {5} 
Number of positions with the same genotype: {4} 
                                                  Heterozigous: {6} 
                                                
                                                  
Number of positions with different genotype: {7} 
                                                  


Total positions compared: {8}
Percentage in common: {4}/{8}= {9}

 ____________________________ END REPORT  _______________________________________
'''
    print(report.format(r0,r1,r2,r3,r4,r5,r6,r7,r8,r9))
    
    # ========== Results ================== #

# To store results in lists and then in df


 ### This is to create the scatter plot with the matched and unmatched samples

# To create the list the first iteration
#sampleA = ["hola"]
#sampleB = ["hola"]
#Total_positions_comp = ["hola"]
#Pos_same_genotype = ["hola"]
#Pos_different_genotyope = ["hola"]
#Hom = ["hola"]
#Het = ["hola"]

#sampleA.append(r1)
#sampleB.append(r3)
#Total_positions_comp.append(r8)
#Pos_same_genotype.append(r4)
#Pos_different_genotyope.append(r7)
#Hom.append(r5)
#Het.append(r6)    


In [ ]:
tmp_df2

In [10]:
def load_sample (Name_file):
    with open(Name_file, 'r') as f:
        for line in f:
            if line.startswith('#') and len(line)>2 and line[1] != '#':
                columns = line[1:-1].split('\t')
                break
    return columns

In [11]:
import argparse                          # pip install argparse
import pandas as pd                      
import os
import numpy as np

In [12]:
NAME_FILE_1 = "/Users/monkiky/Desktop/VCF-matcher/Twin_brothers/W1717693_S1.vcf"

In [13]:
load_sample (NAME_FILE_1)

['CHROM',
 'POS',
 'ID',
 'REF',
 'ALT',
 'QUAL',
 'FILTER',
 'INFO',
 'FORMAT',
 'W1717693']

In [14]:
dataA = pd.read_csv(NAME_FILE_1, comment='#', delimiter='\t', names=load_sample (NAME_FILE_1))
dataA

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,W1717693
0,chr1,36931706,.,T,G,32.0,SB,"DP=1055;TI=NM_172313,NM_156039,NM_000760;GI=CS...",GT:GQ:AD:VF:NL:SB:GQX,"0/1:32:1031,23:0.0218:20:0.0000:32"
1,chr1,36932103,.,T,G,62.0,SB,"DP=2094;TI=NM_172313,NM_156039,NM_000760;GI=CS...",GT:GQ:AD:VF:NL:SB:GQX,"0/1:62:2031,47:0.0224:20:0.0000:62"
2,chr1,36932108,.,A,C,43.0,SB,"DP=2136;TI=NM_172313,NM_156039,NM_000760;GI=CS...",GT:GQ:AD:VF:NL:SB:GQX,"0/1:43:2083,42:0.0197:20:0.0000:43"
3,chr1,36932121,.,C,G,65.0,SB,"DP=321;TI=NM_172313,NM_156039,NM_000760;GI=CSF...",GT:GQ:AD:VF:NL:SB:GQX,"0/1:65:305,16:0.0498:20:-0.2685:65"
4,chr1,36932122,.,T,G,100.0,SB,"DP=333;TI=NM_172313,NM_156039,NM_000760;GI=CSF...",GT:GQ:AD:VF:NL:SB:GQX,"0/1:100:304,26:0.0781:20:-0.2588:100"
...,...,...,...,...,...,...,...,...,...,...
454,chrX,133527688,.,G,GT,100.0,R8,"DP=1090;TI=NM_032335,NM_001015877,NM_032458;GI...",GT:GQ:AD:VF:NL:SB:GQX,"0/1:100:1000,90:0.0826:20:-100.0000:100"
455,chrX,133527928,.,AT,A,50.0,PASS,"DP=1395;TI=NM_032335,NM_001015877,NM_032458;GI...",GT:GQ:AD:VF:NL:SB:GQX,"0/1:50:1362,33:0.0237:20:-22.4949:50"
456,chrX,133547693,.,TA,T,100.0,PASS,"DP=4576;TI=NM_032335,NM_001015877,NM_032458;GI...",GT:GQ:AD:VF:NL:SB:GQX,"0/1:100:4396,180:0.0393:20:-100.0000:100"
457,chrX,133547693,.,T,TA,100.0,PASS,"DP=4493;TI=NM_032335,NM_001015877,NM_032458;GI...",GT:GQ:AD:VF:NL:SB:GQX,"0/1:100:4368,125:0.0278:20:-100.0000:100"


In [15]:
def take_sample(data,NAME_FILE):
    '''
    VCF files can have 1 or > 1 samples
    We need to know what sample the user wants to compare
    After this, the data of the sample is all staored in the same  column
    to access the data, it needs to be splited in different columns
    '''
    if len(data.columns) > 10:
        Name_sample = input('It seems that {} has more than one sample. Please introduce the name of the sample you wish to analyse:  '.format(os.path.basename(NAME_FILE)))
    else:
        print('File', os.path.basename(NAME_FILE), 'contains one sample only.')
        Name_sample = data.columns.tolist()[9]
    
    tmp_df_2 = data[Name_sample].str.split(':', expand=True)
    # I have seen very unlikely Na values
    # To solve this, they are deleted
    tmp_df_2 = tmp_df_2.fillna(value=np.nan)
    tmp_df_2 = tmp_df_2.dropna(axis=1, how='any')
    # To get the name of the new columns created
    tmp_df_2.columns = data.FORMAT.iloc[0].split(':')
    return pd.concat([data, tmp_df_2], axis=1),Name_sample
    



In [16]:
dataA,Name_sample_1 = take_sample(dataA,NAME_FILE_1)


File W1717693_S1.vcf contains one sample only.


In [22]:
def filter_1(data):
    return data.query('FILTER == "PASS"')



In [25]:
def filter_2(data):
    if "VF" in data.columns:
        data["VF"] = data["VF"].astype(float)
    return data.query('VF > 0.4')

In [26]:
dataA = filter_2(dataA)

In [30]:
def comparason(data):
    return dataA["CHROM"].apply(str)+"."+dataA["POS"].apply(str)+dataA["REF"]+"."+dataA["ALT"]


In [31]:
dataA['CHROMPOSREFALT'] = comparason(dataA)

<ipython-input-31-61ab00b914a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataA['CHROMPOSREFALT'] = comparason(dataA)


In [33]:
frames = [dataA[['CHROMPOSREFALT', 'GT']],dataA[['CHROMPOSREFALT', 'GT']]]

In [34]:
semi_final_df = pd.concat(frames)
semi_final_df

,CHROMPOSREFALT,GT
10,chr1.36933096T.C,0/1
16,chr2.25463483G.A,0/1
19,chr2.25466888G.T,1/1
30,chr2.25469502C.T,0/1
33,chr2.198266834T.C,0/1
...,...,...
391,chrX.76940534A.G,1/1
423,chrX.129147079T.C,1/1
424,chrX.129147373G.A,1/1
439,chrX.129186012C.T,1/1


In [ ]:
# To see is lists are storing data correctly
print(sampleA)
print(sampleB)
print(Total_positions_comp)
print(Pos_same_genotype)
print(Pos_different_genotyope)
print(Hom)
print(Het)

In [ ]:
# Save lists in a df
results = pd.DataFrame(list(zip(sampleA, sampleB,Total_positions_comp,Pos_same_genotype,Pos_different_genotyope,Hom,Het)),
               columns=['Sample_A','Sample_B','Total_positions_comp','Pos_same_genotype','Pos_different_genotyope','Hom','Het'])
results = results[1:]


In [ ]:
# To get proportion of sites with common genotype
results["identical_position (%)"] = results["Pos_same_genotype"]/results["Total_positions_comp"]

In [ ]:
# To save 
total_results = pd.read_csv("/Users/monkiky/Desktop/VCF-matcher/Samples/Same_patients/Results/both_patients.csv")
total_results

In [ ]:
# To generater the plot
import seaborn as sns

sns.set_context('notebook')
sns.scatterplot(data=total_results, x="identical_position (%)", y="Total_positions_comp",
               hue = "Samples:",
               palette=['red',"dodgerblue"], legend='full',
               ).set(xlabel="Proportion of positions with common genotypes",
                    ylabel="Number of positions compared")

In [ ]:

################### NOOO DELETE THIS ######################################
# A Dashboard to show results in case I will needed


### 6. Generate a report in a dashboard
# Bootstrap themes by Ann: https://hellodash.pythonanywhere.com/theme_explorer
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SIMPLEX])

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src='./assets/wrgllogohighres.jpeg')
                ],color="light", outline=True),
                ],width=3),
                dbc.Col([
                    dbc.Card([
                        dbc.CardImg(src='./assets/salisbury-nhs-foundation-trustLeft.png')
                             ],color="light", outline=True)
                        ],width=2)
            ],justify="between"),
        dbc.Row(
            [dbc.Card([ 
                dbc.CardBody([
                    html.Div([
                    html.H1("Report: Do the samples belong to the same patient?"),
                    html.H3("This application takes common variants and compares the GT fields. A relative large number of common variants which GT fields match is indicative that both samples have a close relationship."),
                ], style={'textAlign': 'center'}) 
                     ], style={'textLeft':'center', })      
                      ], style={'height':'10vh'},color="light", outline=True),
        ]),
        dbc.Row([
            dbc.Card([ 
                dbc.CardBody([
                    html.Div([
                    html.H1(""),
                    html.H3(""),
                ], style={'textAlign': 'center'}) 
                     ], style={'textLeft':'center', })      
                      ], style={'height':'10vh'},color="light", outline=True),
        ]),
        dbc.Row([  
            dbc.Col([
            dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": True, "selectable": True, "hideable": True}
            if i == "iso_alpha3" or i == "year" or i == "id"
            else {"name": i, "id": i, "deletable": True, "selectable": True}
            for i in final_df.columns
        ],
        data=final_df.to_dict('records'),  # the contents of the table html.H3(os.path.basename(Name_file1)),
        filter_action="native",     # allow filtering of data by user ('native') or not ('none')
        sort_action="native",       # enables data to be sorted per-column by user or not ('none')
        sort_mode="single",         # sort across 'multi' or 'single' columns
        row_deletable=True,         # choose if user can delete a row (True) or not (False)
        selected_rows=[],           # indices of rows that user selects
        page_action="native",       # all data is passed to the table up-front or not ('none')
        page_current=0,             # page number that user is on
        style_cell={                # ensure adequate header width when text is shorter than cell's text
            'minWidth': 200, 'maxWidth': 95, 'width': 95
        },
        style_data_conditional=[
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(248, 248, 248)'
        }
    ],
        style_header={
        'backgroundColor': 'rgb(230, 230, 230)',
        'fontWeight': 'bold'
    })]),
    dbc.Col([
        dbc.Card([ 
            dbc.CardBody([
                html.Div([
                    html.H4("Number of common variants:",style={'color': 'black', 'fontSize': 15}),
                    html.H4(len(final_df)),
                    html.H4("Number of matches:", style={'color': 'black', 'fontSize': 15}),
                    html.H4(final_df.Matches.value_counts()[1]),
                    ], 
                    style={'textAlign': 'left'})
                    ])
                    ],
                    style={'height':'8vh'},color="light", outline=True),
                    ])
                    ])
                    ])



  


if __name__ == '__main__':
    app.run_server(debug=False)




In [26]:
#!/usr/bin/env python
'''
Created on 07/09/2021

@author: manuel.dominguezbecerra@nhs.net

'''

# Import libraries
import os
import argparse
import pandas as pd
import numpy as np

# To allow the user to introduce the file after running "python run.ppy"

# Example:    python run.py --vcf1 /path/file_name1 --vcf2 /path/file_name2

   ### 1. Load the files

# Take only the data contains in the body of the vcf files.

def load_sample (Name_file):
    '''
    Take the header of the body of the CSV file
    '''
    with open(Name_file, 'r') as f:
        for line in f:
            if line.startswith('#') and len(line)>2 and line[1] != '#':
                columns = line[1:-1].split('\t')
                data = pd.read_csv(Name_file, comment='#', delimiter='\t', names=columns)
                break
    return data

    ### 2. One or > one sample in the files? 

def take_sample(data,NAME_FILE):
    '''
    VCF files can have 1 or > 1 samples
    We need to know what sample the user wants to compare
    After this, the data of the sample is all stored in one  column
    to access the data, it needs to  split it in different columns
    '''
    if len(data.columns) > 10:
        Name_sample = input('It seems that {} has more than one sample. Please introduce the name of the sample you wish to analyse:  '.format(os.path.basename(NAME_FILE)))
    else:
        print('File', os.path.basename(NAME_FILE), 'contains one sample only.')
        Name_sample = data.columns.tolist()[9]
    
    tmp_df_2 = data[Name_sample].str.split(':', expand=True)
    # I have seen very unlikely Na values
    # To solve this, they are deleted
    tmp_df_2 = tmp_df_2.fillna(value=np.nan)
    tmp_df_2 = tmp_df_2.dropna(axis=1, how='any')
    # To get the name of the new columns created
    tmp_df_2.columns = data.FORMAT.iloc[0].split(':')
    return pd.concat([data, tmp_df_2], axis=1),Name_sample
    
    ### 3. Filter the variants

def filter_1(data):
    '''
    Take only the variants that "PASS" the "FILTER" field
    '''
    return data.query('FILTER == "PASS"')


def filter_2(data):
    '''
    Take the variants which VF > 0.4
    '''
    if "VF" in data.columns:
        data["VF"] = data["VF"].astype(float)
        return data.query('VF > 0.4')
    else:
        return data

   ### 4. Variants comparison

def Unit_identifier(data):
    '''
    Two variants are the same if  CHROM, POS, REF, ALT and GT are equal
    Here, the unit identifier is created
    '''
    return data["CHROM"].apply(str)+"."+data["POS"].apply(str)+data["REF"]+"."+data["ALT"]


def concatenate_samples(data1,data2):
    '''
    To proceeded, both data frames need to be concatenate 
    '''
    frames = [data1[['CHROMPOSREFALT', 'GT']],data2[['CHROMPOSREFALT', 'GT']]]
    return pd.concat(frames)

def match_variants(data):
    '''
    Match common variants in one column and put the GT values of both samples
    in different columns.
    No common variants are also set in the first column giving NaN values 
    in the third column. I have solved this by substituting the NaN for 99/99 values
    '''
    matched_df = (data.assign(key=data.groupby('CHROMPOSREFALT').cumcount())
        .pivot('CHROMPOSREFALT','key','GT')
        .rename(columns=lambda x:f"Sample{x+1}")
        .rename_axis(columns=None).reset_index())
    matched_df = matched_df.replace(np.NaN,"99/99" )
    return matched_df.iloc[:, : 3]

def count_hom_het_variants(data):
    data['Genotype1'] = data.iloc[:,2].apply(lambda x: x.split('/' or '|')[0])
    data['Genotype2'] = data.iloc[:,2].apply(lambda x: x.split('/' or '|')[1])
    return np.select([data['Matches'] & data['Genotype1'].eq(data['Genotype2']),
                       data['Matches'] & data['Genotype1'].ne(data['Genotype2'])],
                       choicelist=["Hom", "Het"],
                       default=pd.NA)


if __name__ == "__main__":
    # execute only if run as a script

    # Enter the path/file names

    NAME_FILE_1 = "/Users/monkiky/Desktop/VCF_matcher/Testing/test_sample.vcf"
    NAME_FILE_2 = "/Users/monkiky/Desktop/VCF_matcher/Testing/test_sample.vcf"

    # Load the files
    dataA = load_sample(NAME_FILE_1)
    dataB = load_sample(NAME_FILE_2)

    # Save now the data of this sample and its name (we will need for the report)
    dataA,Name_sample_1 = take_sample(dataA,NAME_FILE_1)
    dataB,Name_sample_2 = take_sample(dataB,NAME_FILE_2)

    # Filter 1
    dataA = filter_1(dataA)
    dataB = filter_1(dataB)

    # Filter 2
    dataA = filter_2(dataA)
    dataB = filter_2(dataB)

    # Unit_identifier
    dataA['CHROMPOSREFALT'] = Unit_identifier(dataA)
    dataB['CHROMPOSREFALT'] = Unit_identifier(dataB)


    # Concatenate both data frames
    semi_final_df = concatenate_samples(dataA,dataB)

    # Match common variants
    final_df= match_variants(semi_final_df)

#### Generate report ####

    # Variables needed for both type of reports

    r0 = os.path.basename(NAME_FILE_1) # Get the  file name of a path/file_name
    r1 = Name_sample_1                 # Get the name of the sample 1
    r2 = os.path.basename(NAME_FILE_2)
    r3 = Name_sample_2

    if "Sample2" not in final_df.columns:
    # If Sample2 is not in final_df, that means there is not common position between the samples.
    # This is a the most simple kind of report with only the name of the files
    # and samples and informing that no common positions have been found between the samples
        REPORT_A = '''
 _____________________________  REPORT  ________________________________________ 

   vcf 1: {0}  AND its sample name: {1}  
   vcf 2:  {2}  AND its sample name: {3}

   No common positions found between samples

 ____________________________ END REPORT  _______________________________________
'''
        print(REPORT_A.format(r0,r1,r2,r3,))

    # If Sample2 is in final_df.columns, that means that at least one common variant has been found
    # so, generate the data from these common variants
    else:
    # Else count common positions and carry on the report.
        final_df["Matches"] = np.where(final_df["Sample1"] == final_df["Sample2"], True, False)
        final_df.columns = ['CHROMPOSREFALT',os.path.basename(NAME_FILE_1),os.path.basename(NAME_FILE_2),"Matches" ]

    #  Get and count hom and het common variants
        final_df['Hom/het'] = count_hom_het_variants(final_df)

    # Generate the second type of report

        r4 = final_df['Matches'].value_counts().get(True, 0) # Count trues if any, return 0
        r5 = final_df['Hom/het'].value_counts().get("Hom", 0)
        r6 =final_df['Hom/het'].value_counts().get("Het", 0)
        r7 = final_df['Matches'].value_counts().get(False, 0)
        r8 = len(final_df)
        r9 = r4/(r4+r7)
        REPORT_B = '''
 _____________________________  REPORT  ________________________________________ 

vcf 1: {0}  AND its sample name: {1}  
vcf 2:  {2}  AND its sample name: {3}

                                                  Homozygous: {5} 
Number of positions with the same genotype: {4} 
                                                  Heterozygous: {6} 
                                                
                                                  
Number of positions with different genotype: {7} 
                                                  


Total positions compared: {8}
Percentage in common: {4}/{8}= {9}

 ____________________________ END REPORT  _______________________________________
'''
        print(REPORT_B.format(r0,r1,r2,r3,r4,r5,r6,r7,r8,r9))


It seems that test_sample.vcf has more than one sample. Please introduce the name of the sample you wish to analyse:  373978487
It seems that test_sample.vcf has more than one sample. Please introduce the name of the sample you wish to analyse:  373978487

 _____________________________  REPORT  ________________________________________ 

vcf 1: test_sample.vcf  AND its sample name: 373978487  
vcf 2:  test_sample.vcf  AND its sample name: 373978487

                                                  Homozygous: 1341 
Number of positions with the same genotype: 1342 
                                                  Heterozygous: 1 
                                                
                                                  
Number of positions with different genotype: 0 
                                                  


Total positions compared: 1342
Percentage in common: 1342/1342= 1.0

 ____________________________ END REPORT  _______________________________________



In [29]:
final_df['Hom/het'].value_counts()["Hom"]

1341

In [23]:
semi_final_df = concatenate_samples(dataA,dataB)
semi_final_df

,CHROMPOSREFALT,GT
0,1.2160224G.A,0/0
1,1.2160661C.T,0/0
2,1.2161003C.T,0/0
3,1.2234488C.T,0/0
4,1.2235378C.T,0/1
...,...,...
1394,X.153588893G.A,0/0
1395,X.153591022G.T,0/0
1396,X.153592948G.A,0/0
1398,X.153594535G.A,0/0


In [24]:
final_df= match_variants(semi_final_df)
final_df

,CHROMPOSREFALT,Sample1,Sample2
0,1.12009955C.T,0/0,0/0
1,1.12009956G.A,0/0,0/0
2,1.12010469G.T,0/0,0/0
3,1.12017959A.G,0/0,0/0
4,1.12024235C.T,0/0,0/0
...,...,...,...
1337,X.595475C.A,0/0,0/0
1338,X.8503641G.A,1/1,1/1
1339,X.8504833C.T,1/1,1/1
1340,X.9707566C.T,0/0,0/0


In [ ]:
#!/usr/bin/env python
'''
Created on 07/09/2021

@author: manuel.dominguezbecerra@nhs.net

'''

# Import libraries
import os
import argparse
import pandas as pd
import numpy as np

# To allow the user to introduce the file after running "python run.ppy"

# Example:    python run.py --vcf1 /path/file_name1 --vcf2 /path/file_name2

   ### 1. Load the files

# Take only the data contains in the body of the vcf files.

def load_sample (Name_file):
    '''
    Take the header of the body of the CSV file
    '''
    with open(Name_file, 'r') as f:
        for line in f:
            if line.startswith('#') and len(line)>2 and line[1] != '#':
                columns = line[1:-1].split('\t')
                data = pd.read_csv(Name_file, comment='#', delimiter='\t', names=columns)
                break
    return data

    ### 2. One or > one sample in the files? 

def take_sample(data,NAME_FILE):
    '''
    VCF files can have 1 or > 1 samples
    We need to know what sample the user wants to compare
    After this, the data of the sample is all stored in one  column
    to access the data, it needs to  split it in different columns
    '''
    if len(data.columns) > 10:
        Name_sample = input('It seems that {} has more than one sample. Please introduce the name of the sample you wish to analyse:  '.format(os.path.basename(NAME_FILE)))
    else:
        print('File', os.path.basename(NAME_FILE), 'contains one sample only.')
        Name_sample = data.columns.tolist()[9]
    
    tmp_df_2 = data[Name_sample].str.split(':', expand=True)
    # I have seen very unlikely Na values
    # To solve this, they are deleted
    tmp_df_2 = tmp_df_2.fillna(value=np.nan)
    tmp_df_2 = tmp_df_2.dropna(axis=1, how='any')
    # To get the name of the new columns created
    tmp_df_2.columns = data.FORMAT.iloc[0].split(':')
    return pd.concat([data, tmp_df_2], axis=1),Name_sample
    
    ### 3. Filter the variants

def filter_1(data):
    '''
    Take only the variants that "PASS" the "FILTER" field
    '''
    return data.query('FILTER == "PASS"')


def filter_2(data):
    '''
    Take the variants which VF > 0.4
    '''
    if "VF" in data.columns:
        data["VF"] = data["VF"].astype(float)
        return data.query('VF > 0.4')
    else:
        return data

   ### 4. Variants comparison

def Unit_identifier(data):
    '''
    Two variants are the same if  CHROM, POS, REF, ALT and GT are equal
    Here, the unit identifier is created
    '''
    return data["CHROM"].apply(str)+"."+data["POS"].apply(str)+data["REF"]+"."+data["ALT"]


def concatenate_samples(data1,data2):
    '''
    To proceeded, both data frames need to be concatenate 
    '''
    frames = [data1[['CHROMPOSREFALT', 'GT']],data2[['CHROMPOSREFALT', 'GT']]]
    return pd.concat(frames)

def match_variants(data):
    '''
    Match common variants in one column and put the GT values of both samples
    in different columns.
    No common variants are also set in the first column giving NaN values 
    in the third column. I have solved this by substituting the NaN for 99/99 values
    '''
    matched_df = (data.assign(key=data.groupby('CHROMPOSREFALT').cumcount())
        .pivot('CHROMPOSREFALT','key','GT')
        .rename(columns=lambda x:f"Sample{x+1}")
        .rename_axis(columns=None).reset_index())
    matched_df = matched_df.replace(np.NaN,"99/99" )
    return matched_df.iloc[:, : 3]

def count_hom_het_variants(data):
    data['Genotype1'] = data.iloc[:,2].apply(lambda x: x.split('/' or '|')[0])
    data['Genotype2'] = data.iloc[:,2].apply(lambda x: x.split('/' or '|')[1])
    return np.select([data['Matches'] & data['Genotype1'].eq(data['Genotype2']),
                       data['Matches'] & data['Genotype1'].ne(data['Genotype2'])],
                       choicelist=["Hom", "Het"],
                       default=pd.NA)

def count_and_report(data):
    r4 = data['Matches'].value_counts().get(True, 0) # Count trues if any, return 0
    r5 = data['Hom/het'].value_counts().get("Hom", 0)
    r6 = data['Hom/het'].value_counts().get("Het", 0)
    r7 = data['Matches'].value_counts().get(False, 0)
    r8 = len(data)
    r9 = r4/(r4+r7)
    return r4,r5,r6,r7,r8,r9 


if __name__ == "__main__":
    # execute only if run as a script

    # Enter the path/file names

    NAME_FILE_1 = "/Users/monkiky/Desktop/Same_patient/W2008872_S8-J4PJL.vcf"
    NAME_FILE_2 = "/Users/monkiky/Desktop/VCF_matcher/Testing/test_sample.vcf"

    # Load the files
    dataA = load_sample(NAME_FILE_1)
    dataB = load_sample(NAME_FILE_2)

    # Save now the data of this sample and its name (we will need for the report)
    dataA,Name_sample_1 = take_sample(dataA,NAME_FILE_1)
    dataB,Name_sample_2 = take_sample(dataB,NAME_FILE_2)

    # Filter 1
    dataA = filter_1(dataA)
    dataB = filter_1(dataB)

    # Filter 2
    dataA = filter_2(dataA)
    dataB = filter_2(dataB)

    # Unit_identifier
    dataA['CHROMPOSREFALT'] = Unit_identifier(dataA)
    dataB['CHROMPOSREFALT'] = Unit_identifier(dataB)

    # Concatenate both data frames
    semi_final_df = concatenate_samples(dataA,dataB)

    # Match common variants
    final_df= match_variants(semi_final_df)

#### Generate report ####

    # Variables needed for both type of reports

    r0 = os.path.basename(NAME_FILE_1) # Get the  file name of a path/file_name
    r1 = Name_sample_1                 # Get the name of the sample 1
    r2 = os.path.basename(NAME_FILE_2)
    r3 = Name_sample_2

    if "Sample2" not in final_df.columns:
    # If Sample2 is not in final_df, that means there is not common position between the samples.
    # This is a the most simple kind of report with only the name of the files
    # and samples and informing that no common positions have been found between the samples
        REPORT_A = '''
 _____________________________  REPORT  ________________________________________ 

   vcf 1: {0}  AND its sample name: {1}  
   vcf 2:  {2}  AND its sample name: {3}

   No common positions found between samples

 ____________________________ END REPORT  _______________________________________
'''
        print(REPORT_A.format(r0,r1,r2,r3,))

    # If Sample2 is in final_df.columns, that means that at least one common variant has been found
    # so, generate the data from these common variants
    else:
    # Else count common positions and carry on the report.
        final_df["Matches"] = np.where(final_df["Sample1"] == final_df["Sample2"], True, False)
        final_df.columns = ['CHROMPOSREFALT',os.path.basename(NAME_FILE_1),os.path.basename(NAME_FILE_2),"Matches" ]

    #  Get and count hom and het common variants
        final_df['Hom/het'] = count_hom_het_variants(final_df)

    # Generate the second type of report
        r4,r5,r6,r7,r8,r9 = count_and_report(final_df)
        REPORT_B = '''
 _____________________________  REPORT  ________________________________________ 

vcf 1: {0}  AND its sample name: {1}  
vcf 2:  {2}  AND its sample name: {3}

                                                  Homozygous: {5} 
Number of positions with the same genotype: {4} 
                                                  Heterozygous: {6} 
                                                
                                                  
Number of positions with different genotype: {7} 
                                                  


Total positions compared: {8}
Percentage in common: {4}/{8}= {9}

 ____________________________ END REPORT  _______________________________________
'''
        print(REPORT_B.format(r0,r1,r2,r3,r4,r5,r6,r7,r8,r9))



File W2008872_S8-J4PJL.vcf contains one sample only.


In [39]:
def load_sample (Name_file):
    '''
    Take the header of the body of the CSV file
    '''
    with open(Name_file, 'r') as f:
        for line in f:
            if line.startswith('#') and len(line)>2 and line[1] != '#':
                columns = line[1:-1].split('\t')
                data = pd.read_csv(Name_file, comment='#', delimiter='\t', names=columns)
                break
    return data

def take_sample(data,NAME_FILE):
    '''
    VCF files can have 1 or > 1 samples
    We need to know what sample the user wants to compare
    After this, the data of the sample is all stored in one  column
    to access the data, it needs to  split it in different columns
    '''
    if len(data.columns) > 10:
        Name_sample = input('It seems that {} has more than one sample. Please introduce the name of the sample you wish to analyse:  '.format(os.path.basename(NAME_FILE)))
    else:
        print('File', os.path.basename(NAME_FILE), 'contains one sample only.')
        Name_sample = data.columns.tolist()[9]
        print(Name_sample)
    
    tmp_df_2 = data[Name_sample].str.split(':', expand=True)
    # I have seen very unlikely Na values
    # To solve this, they are deleted
    tmp_df_2 = tmp_df_2.fillna(value=np.nan)
    tmp_df_2 = tmp_df_2.dropna(axis=1, how='any')
    # To get the name of the new columns created
    tmp_df_2.columns = data.FORMAT.iloc[0].split(':')
    print(tmp_df_2)
    return pd.concat([data, tmp_df_2], axis=1),Name_sample

In [48]:
# Load the files
dataA = load_sample(NAME_FILE_2)
dataA,Name_sample_1 = take_sample(dataA,NAME_FILE_1)

File W2008872_S8-J4PJL.vcf contains one sample only.


IndexError: list index out of range

In [50]:
load_sample(NAME_FILE_2)

,Import all functions from run.py
0,'''
1,This script breaks down fragments of the file ...
2,to carry out different tests.
3,A large vcf file has been create manually to t...
4,'''
...,...
130,assert r6 == 1
131,assert r7 == 0
132,assert r8 == 1342
133,assert r9 == 1.0
